In [2]:
import requests
import json
from bs4 import BeautifulSoup
from urllib.parse import urljoin, quote
from lxml import etree
import selenium
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
import time
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import datetime
from selenium.webdriver.edge.options import Options

In [3]:
def generate_urls(jugador, anio_inicio, mes_inicio, dia_inicio, anio_fin, mes_fin, dia_fin):
    # referencia https://news.google.com/search?q=nombre_jugador+de_futbol+after:YYYY-MM-DD+before:YYYY-MM-DD
    
    interval_tuples = []
    
    # se hacen intervalos de 1 semana
    while True:
        if anio_inicio == anio_fin and mes_inicio == mes_fin and dia_inicio >= dia_fin:
            break

        if anio_inicio == anio_fin and mes_inicio > mes_fin and dia_inicio:
            break

        if anio_inicio > anio_fin:
            break
        
        interval_tuples.append((anio_inicio, mes_inicio, dia_inicio))
        
        dia_inicio += 7
        
        if dia_inicio > 30:
            dia_inicio = 1
            mes_inicio += 1
        
        if mes_inicio > 12:
            mes_inicio = 1
            anio_inicio += 1

    print(interval_tuples)

    urls = []

    for interval in interval_tuples:
        anio, mes, dia = interval

        url = f"https://news.google.com/search?q={jugador.replace(' ', '%20')}+after:{anio}-{mes}-{dia}+before:{anio}-{mes}-{dia + 7}"
        urls.append(url)


    return urls


In [4]:
def get_news_links(url):
    edge_options = Options()
    edge_options.add_argument('--headless')
    edge_options.add_argument('--disable-gpu')
    edge_options.add_argument('--no-sandbox') 
    
    driver = webdriver.Edge(options=edge_options)
    driver.get(url)

    # Espera a que el botón de 'Accept all' esté presente y clicable
    wait = WebDriverWait(driver, 10)  # Espera hasta 10 segundos
    accept_button = wait.until(EC.element_to_be_clickable((By.XPATH, "//span[text()='Aceptar todo']")))

    # Hace clic en el elemento <span>
    accept_button.click()

    # Esperar a que la nueva página comience a cargarse o hacer algo en el medio si es necesario
    time.sleep(2)  # Pausa opcional para dar tiempo a la carga inicial de la página

    # Scroll hacia abajo hasta el final de la página
    last_height = driver.execute_script("return document.body.scrollHeight")

    while True:
            # Desplazarse hacia abajo
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            
            # Esperar a que se cargue la nueva sección
            time.sleep(2)  # Ajusta el tiempo según sea necesario para la página
            
            # Calcular la nueva altura de la página después del scroll
            new_height = driver.execute_script("return document.body.scrollHeight")
            
            # Si la altura no cambia, hemos llegado al final de la página
            if new_height == last_height:
                break
            
            last_height = new_height

    elements = driver.find_elements(By.CSS_SELECTOR, 'c-wiz[jsrenderer="ARwRbe"]')

    # Extraer los href de cada elemento si tienen un enlace dentro
    links = []
    for element in elements:
        # Buscar el enlace dentro del elemento
        link = element.find_element(By.TAG_NAME, 'a')
        href = link.get_attribute('href')
        if href:
            links.append(href)





    # Encontrar todos los elementos <time> con class="hvbAAd"
    time_elements = driver.find_elements(By.CSS_SELECTOR, 'time.hvbAAd')

    # Extraer el valor del atributo 'datetime' de cada <time>
    datetimes = []
    for element in time_elements:
        datetime_value = element.get_attribute('datetime')
        if datetime_value:
            datetimes.append(datetime_value)







    driver.quit()

    return links[1:], datetimes





In [5]:
def get_news_parragraphs(url):
    edge_options = Options()
    edge_options.add_argument('--headless')

    driver = webdriver.Edge(options=edge_options)
    driver.get(url)


    try:
        # Espera a que el botón de 'Accept all' esté presente y clicable
        wait = WebDriverWait(driver, 10)  # Espera hasta 10 segundos
        accept_button = wait.until(EC.element_to_be_clickable((By.XPATH, "//span[text()='Accept all']")))
        
        # Hace clic en el elemento <span>
        accept_button.click()

        # Esperar a que la nueva página comience a cargarse o hacer algo en el medio si es necesario
        time.sleep(2)  # Pausa opcional para dar tiempo a la carga inicial de la página

        # Scroll hacia abajo hasta el final de la página
        last_height = driver.execute_script("return document.body.scrollHeight")
        
        while True:
            # Desplazarse hacia abajo
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            
            # Esperar a que se cargue la nueva sección
            time.sleep(2)  # Ajusta el tiempo según sea necesario para la página
            
            # Calcular la nueva altura de la página después del scroll
            new_height = driver.execute_script("return document.body.scrollHeight")
            
            # Si la altura no cambia, hemos llegado al final de la página
            if new_height == last_height:
                break
            
            last_height = new_height
        
        # Una vez que se ha llegado al final, obtener todos los párrafos <p>
        paragraphs = driver.find_elements(By.TAG_NAME, 'p')
        
        # Imprimir el texto de cada párrafo
        for paragraph in paragraphs:
            return paragraph.text

        
    except Exception as e:
        print(f"Error: {e}")
    finally:
        driver.quit()


In [6]:
root_urls = generate_urls("Vinicius Jr", 2022, 1, 1, 2022, 6, 1)

news_links_list = []
news_datetimes_list = []

for url in root_urls:
    news_links, news_datetimes = get_news_links(url)
    news_links_list.extend(news_links)
    news_datetimes_list.extend(news_datetimes)

list_paragraphs = []

for news_links in news_links_list:
    paragraph = get_news_parragraphs(news_links)
    list_paragraphs.append(paragraph)

[(2022, 1, 1), (2022, 1, 8), (2022, 1, 15), (2022, 1, 22), (2022, 1, 29), (2022, 2, 1), (2022, 2, 8), (2022, 2, 15), (2022, 2, 22), (2022, 2, 29), (2022, 3, 1), (2022, 3, 8), (2022, 3, 15), (2022, 3, 22), (2022, 3, 29), (2022, 4, 1), (2022, 4, 8), (2022, 4, 15), (2022, 4, 22), (2022, 4, 29), (2022, 5, 1), (2022, 5, 8), (2022, 5, 15), (2022, 5, 22), (2022, 5, 29)]
Error: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF753B6CEC5+12997]
	Microsoft::Applications::Events::EventProperty::empty [0x00007FF753DEBCA4+1897908]
	(No symbol) [0x00007FF75391F1CC]
	(No symbol) [0x00007FF75396672E]
	(No symbol) [0x00007FF7539667B5]
	(No symbol) [0x00007FF7539A4ED7]
	(No symbol) [0x00007FF753987F8F]
	(No symbol) [0x00007FF75395C09D]
	(No symbol) [0x00007FF7539A2887]
	(No symbol) [0x00007FF753987BB3]
	(No symbol) [0x00007FF75395B5AC]
	(No symbol) [0x00007FF75395AA8D]
	(No symbol) [0x00007FF75395B171]
	Microsoft::Applications::Events::EventProperty::empty [0x00007FF753D19DD4+1038052]
	Microsoft::Appli

KeyboardInterrupt: 